In [53]:
print("running")

running


In [68]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import pickle

In [56]:
df=pd.read_csv("/Users/yashrajput/Desktop/customer segmentation app/DataSets/churn.csv")
# df.head()

In [57]:
df.drop(columns=['Unnamed: 0', 'security_no', 'referral_id', 'joining_date', 'last_visit_time', 'feedback'], inplace=True)
# df.columns
# df.isnull().sum()

age                                0
gender                             0
region_category                 5428
membership_category                0
joined_through_referral            0
preferred_offer_types            288
medium_of_operation                0
internet_option                    0
days_since_last_login              0
avg_time_spent                     0
avg_transaction_value              0
avg_frequency_login_days           0
points_in_wallet                3443
used_special_discount              0
offer_application_preference       0
past_complaint                     0
complaint_status                   0
churn_risk_score                   0
dtype: int64

In [58]:
# Step 1: Handle missing values
# Fill missing values for categorical columns with 'Unknown' and numerical columns with median
df['region_category'].fillna('Unknown', inplace=True)
df['preferred_offer_types'].fillna('Unknown', inplace=True)
df['points_in_wallet'].fillna(df['points_in_wallet'].median(), inplace=True)

In [59]:
# Step 2: Convert 'avg_frequency_login_days' to numeric, coercing errors to NaN and filling with median
df['avg_frequency_login_days'] = pd.to_numeric(df['avg_frequency_login_days'], errors='coerce')
df['avg_frequency_login_days'].fillna(df['avg_frequency_login_days'].median(), inplace=True)

In [60]:
# Step 3: Encode categorical columns
le = LabelEncoder()
categorical_columns = ['gender', 'region_category', 'membership_category', 'joined_through_referral', 
                       'preferred_offer_types', 'medium_of_operation', 'internet_option', 'used_special_discount', 
                       'offer_application_preference', 'past_complaint', 'complaint_status']

for col in categorical_columns:
    df[col] = le.fit_transform(df[col])

In [61]:
# Step 4: Separate features and target variable
X = df.drop('churn_risk_score', axis=1)  # Features
y = df['churn_risk_score']               # Target


In [62]:
# Step 5: Scaling numerical features
scaler = StandardScaler()
numerical_columns = ['age', 'days_since_last_login', 'avg_time_spent', 
                     'avg_transaction_value', 'avg_frequency_login_days', 'points_in_wallet']

X[numerical_columns] = scaler.fit_transform(X[numerical_columns])

In [63]:
# Step 6: Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [64]:
# Step 7: Train the Model
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [69]:
# Step 8: Ensure the 'models/' directory exists
os.makedirs('models', exist_ok=True)

# Save the model
with open('models/churn_model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model saved successfully!")


Model saved successfully!


In [70]:
# Step 9: Evaluate the Model
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Accuracy: 0.761319097175294
              precision    recall  f1-score   support

           0       0.76      0.70      0.73      3377
           1       0.76      0.81      0.79      4022

    accuracy                           0.76      7399
   macro avg       0.76      0.76      0.76      7399
weighted avg       0.76      0.76      0.76      7399

